In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import logging
import numpy as np
import os

logger = logging.getLogger(__name__)
figsize = (8, 4)


def plot_metrics(df, metrics=None, title="Pool Metrics", y_range=None):
    if metrics is None:
        metrics = [
            ("pool_virtual_price", "tab:blue", "-", "virtual_price"),
            ("pool_xcp_profit", "tab:orange", "-", "xcp_profit"),
            ("pool_xcp_profit_a", "tab:red", "-", "xcp_profit_a"),
            ("pool_xcpx", "tab:green", "-", "xcpx"),
            ("pool_xcp_half", "tab:purple", "-", "xcp_half"),
        ]
    fig, ax = plt.subplots(figsize=figsize)
    x_points = df["iteration"]
    all_values = {metric_name: df["data_" + metric_name] for metric_name, _, _, _ in metrics}
    for metric_name, color, linestyle, label in metrics:
        ax.plot(
            x_points,
            all_values[metric_name],
            label=label,
            color=color,
            linestyle=linestyle,
            alpha=0.8,
            linewidth=1,
        )
    # plot vertical lines where rebalance_happened is True
    for i, row in df.iterrows():
        if row["rebalance_happened"]:
            ax.axvline(x=i + 1, color="red", linestyle="--", alpha=0.3, linewidth=1)
        if row["fees_claimed"]:
            ax.axvline(x=i + 1, color="green", linestyle="--", alpha=0.3, linewidth=1)
    ax.set_ylabel("Metric Value")
    ax.set_title(title)
    ax.grid(True, linestyle="-", alpha=0.7)
    ax.legend(loc="best")
    if y_range:
        ax.set_ylim(y_range)
    fig.tight_layout()
    plt.show()
    return fig


def plot_total_values(df, actors=None, title="Total Values", y_range=None):
    fig, ax = plt.subplots(figsize=figsize)
    actors_all = [
        ("pool", "tab:purple", "Pool"),
        ("lp_user", "tab:blue", "LP"),
        ("trader", "tab:orange", "Trader"),
        ("admin", "tab:green", "Admin"),
    ]
    if actors is None:
        actors = actors_all
    else:
        actors = [a for a in actors_all if a[0] in actors]
    # total = np.zeros(len(df))
    for actor, color, label in actors:
        col = f"data_{actor}_total_value"
        ax.plot(df["iteration"], df[col], label=label, color=color, alpha=0.8)
    # total = df['data_pool_total_value'] + df['data_admin_total_value']
    # ax.plot(df["iteration"], total, label="Pool+Admin", color="tab:red", alpha=0.8)
    ax.set_xlabel("Iteration")
    ax.set_ylabel("Total Value (in coin0 terms)")
    ax.set_title(title)
    ax.grid(True, linestyle="--", alpha=0.7)
    ax.legend(loc="best")
    if y_range:
        ax.set_ylim(y_range)
    fig.tight_layout()
    plt.show()
    return fig

In [ ]:
# Example usage
cur_dir = os.path.dirname(os.path.abspath(".")) + "/simulator/sim_data"
# list cur dir and take latest timestamped file
# last_file = sorted(os.listdir(cur_dir))[-1]
csv_filename = os.path.join(cur_dir, "sim_data.csv")
df = pd.read_csv(csv_filename)
_ = plot_metrics(df, title="sim_data")
_ = plot_total_values(df)

In [ ]:
# Example usage
cur_dir = os.path.dirname(os.path.abspath(".")) + "/simulator/sim_data"
# list cur dir and take latest timestamped file
# last_file = sorted(os.listdir(cur_dir))[-1]
csv_1 = os.path.join(cur_dir, "sim_data_1.csv")
csv_2 = os.path.join(cur_dir, "sim_data_2.csv")

df_1 = pd.read_csv(csv_1)
df_2 = pd.read_csv(csv_2)

fig = plot_metrics(df_1, title=csv_1.split("/")[-1])
# fetch ylim from plot
y_range = fig.gca().get_ylim()
_ = plot_metrics(df_2, title=csv_2.split("/")[-1], y_range=y_range)

fig = plot_total_values(df_2, actors=["admin"], title=csv_2.split("/")[-1])
y_range = fig.gca().get_ylim()
_ = plot_total_values(df_1, actors=["admin"], title=csv_1.split("/")[-1], y_range=y_range)

In [ ]:
plt.plot(df["iteration"], df["washtrade_pool_D"], label="ps")

In [ ]:
plt.plot(df["iteration"], df["washtrade_pool_virtual_price"], label="wash")
plt.plot(df["iteration"], df["remove_liquidity_pool_virtual_price"], label="remove")
plt.legend()
plt.show()

In [ ]:
x = 1_000_000
y = x // 1_500  # some init price
price_scale = np.arange(1_000, 2_000)
total_value = x + y * price_scale
value_1 = np.repeat(x, len(price_scale))
value_2 = y * price_scale
plt.plot(price_scale, total_value, label="total")
plt.plot(price_scale, value_1, label="coin0")
plt.plot(price_scale, value_2, label="coin1")
plt.legend()
plt.show()

In [ ]:
import plotly.graph_objs as go
import numpy as np

D = np.arange(1_000, 2_000)
price_scale = np.arange(1_000, 2_000)
D_grid, price_scale_grid = np.meshgrid(D, price_scale)
xcp_grid = D_grid / 2 / np.sqrt(price_scale_grid)

fig = go.Figure(data=[go.Surface(z=xcp_grid, x=D_grid, y=price_scale_grid)])
fig.update_layout(
    scene=dict(
        xaxis_title="D",
        yaxis_title="price_scale",
        zaxis_title="xcp",
        camera=dict(
            eye=dict(x=2, y=2, z=2)  # Increase these values to zoom out
        ),
    )
)
# fit plot to window
fig.update_layout(
    width=500,
    height=500,
    margin=dict(l=0, r=0, t=0, b=0),
    autosize=True,
)


fig.show()